In [14]:
import csv
import json

# Read CSV file
csv_file = 'chicago neighborhoods - v2.csv'  
json_file = 'neighborhood_data.json'

data = []

with open(csv_file, mode='r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        data.append({
            "neighborhood": row["Neighborhood"],
            "community_area": row["Community Area"],
            "region": row["Region"]
        })

# Write to JSON file
with open(json_file, mode='w') as file:
    json.dump(data, file, indent=4)


In [15]:
# Load GeoJSON file
with open('Boundaries - Neighborhoods.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)

# Load Neighborhood Data from the CSV-to-JSON conversion
with open('neighborhood_data.json', 'r') as csv_json_file:
    neighborhood_data = json.load(csv_json_file)

# Create a dictionary for fast lookup using the 'neigh' as the key
neighborhood_dict = {n['neighborhood']: n for n in neighborhood_data}

# Update the GeoJSON properties
for feature in geojson_data['features']:
    pri_neigh = feature['properties'].get('pri_neigh')

    # Check if the neighborhood exists in our CSV data
    if pri_neigh in neighborhood_dict:
        feature['properties']['neighborhood'] = neighborhood_dict[pri_neigh]['neighborhood']
        feature['properties']['community_area'] = neighborhood_dict[pri_neigh]['community_area']
        feature['properties']['region'] = neighborhood_dict[pri_neigh]['region']

# Save the updated GeoJSON
with open('Updated_Boundaries_Neighborhoods.geojson', 'w') as output_file:
    json.dump(geojson_data, output_file, indent=4)


In [16]:
# Create a set of neighborhood names from the CSV (neigh field)
csv_neighborhoods = set(n['neighborhood'] for n in neighborhood_data)

# List to hold unmatched pri_neigh values
unmatched_neighborhoods = []

# Iterate through the GeoJSON features
for feature in geojson_data['features']:
    pri_neigh = feature['properties'].get('pri_neigh')

    # If the pri_neigh is not in the CSV's neighborhood list, add it to unmatched list
    if pri_neigh not in csv_neighborhoods:
        unmatched_neighborhoods.append(pri_neigh)

# Print the unmatched pri_neigh values
print("Unmatched pri_neigh values:")
for neighborhood in unmatched_neighborhoods:
    print(neighborhood)

Unmatched pri_neigh values:
